In [2]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime
from astropy.coordinates import SkyCoord
import pickle, json, sys, os, glob
import pandas as pd
pd.set_option('display.max_columns', None)

# location of the scripts
sys.path.insert(0, os.path.join("/fefs/aswg/workspace/juan.jimenez/cosmic_ray_data_correction/scripts"))
import auxiliar as aux
import documents as docs

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# =================================== #
# paths 
folder_dict_sed_lc = "objects/dicts_sed_and_lc"
fname_dict_dl2     = "objects/dict_dl2.pkl"

folder_total_dicts = "objects/total_dicts"

energy_ranges_info = "bash_energy_ranges/jobs_information.txt"
# =================================== #


# =================================== #
#         intensity profiles
# =================================== #

# intensity binning
binsI = np.linspace(1.1, 4, 200) 

# =================================== #

# load dl2 data
with open(fname_dict_dl2, "rb") as f:
    dict_dl2 = pickle.load(f)
    
# create folder t do not exist
aux.createdir(folder_total_dicts)

# extracting LC limits
a_lcs = [np.array(s.split(",")).astype(float) for s in np.loadtxt(energy_ranges_info, dtype=str)]

In [4]:
%%time
for fname_dict_sed_lc in np.sort(glob.glob(folder_dict_sed_lc + "/*")):

    # Energy LC limits
    totfname = fname_dict_sed_lc.split("/")[-1].split("t_")[1].split(".p")[0]
    fname_dict = os.path.join(folder_total_dicts, f"dict_total_{totfname}.pkl")
    
    if not os.path.exists(fname_dict):

        print(f"Creating total dict --> dict_total_{totfname}.pkl")

        # light curve data
        with open(fname_dict_sed_lc, "rb") as f:
            dict_gammapy = pickle.load(f)

        dict_LC_run = {}
        for i, run in enumerate(np.sort(np.array(dict_gammapy["lightcurve"]["run_number"]).astype(int))):

            tmp_dict = {

                "flux"      : dict_gammapy["lightcurve"]["flux"][i]   * 1e10,  # cm-2 s-1
                "e_flux"    : dict_gammapy["lightcurve"]["e_flux"][i] * 1e10,  # cm-2 s-1
                "timestamp" : dict_gammapy["lightcurve"]["t_start"][i],
                "duration"  : dict_gammapy["lightcurve"]["timedelta"][i]

            }

            dict_LC_run[run] = tmp_dict

        dict_total = {}

        dict_total["general"] = {

            "target_name"   : dict_gammapy["lightcurve"]["global"]["target_name"],
            "e_min_lc"      : dict_gammapy["lightcurve"]["global"]["e_min"], # TeV
            "e_max_lc"      : dict_gammapy["lightcurve"]["global"]["e_max"], # TeV
            "n_off_regions" : dict_gammapy["lightcurve"]["global"]["n_off_regions"],
            "chi2_flux"     : dict_gammapy["lightcurve"]["global"]["chi2"],
            "pvalue_flux"   : dict_gammapy["lightcurve"]["global"]["pvalue"],
            "crab_reference_flux" : dict_gammapy["lightcurve"]["global"]["crab_reference_flux"].value * 1e10   

        }

        _ampl, _e_ampl = dict_gammapy["dict_model"]["spectral"]["parameters"][0]["value"], dict_gammapy["dict_model"]["spectral"]["parameters"][0]["error"]
        _ampl_unit     = u.Unit(dict_gammapy["dict_model"]["spectral"]["parameters"][0]["unit"])

        _ref      = dict_gammapy["dict_model"]["spectral"]["parameters"][1]["value"]
        _ref_unit = u.Unit(dict_gammapy["dict_model"]["spectral"]["parameters"][1]["unit"])

        _a, _e_a = dict_gammapy["dict_model"]["spectral"]["parameters"][2]["value"], dict_gammapy["dict_model"]["spectral"]["parameters"][2]["error"]
        _b, _e_b = dict_gammapy["dict_model"]["spectral"]["parameters"][3]["value"], dict_gammapy["dict_model"]["spectral"]["parameters"][3]["error"]

        dict_total["general"]["model_params"] = {

                "amplitude" : (_ampl, _e_ampl) * _ampl_unit,
                "reference" :  _ref * _ref_unit,
                "alpha"     : (_a, _e_a),
                "beta"      : (_b, _e_b)

        }

        dict_total["general"]["SED"] = dict_gammapy["table_sed"]


        dict_total["run"] = dict_LC_run

        for run in np.sort(list(dict_dl2.keys())):

            dict_total["run"][run]["zd"]   = np.mean(dict_dl2[run]["zd"])
            dict_total["run"][run]["e_zd"] = (np.max(dict_dl2[run]["zd"]) - np.min(dict_dl2[run]["zd"])) / 2
            dict_total["run"][run]["az"]   = np.mean(dict_dl2[run]["az"])
            dict_total["run"][run]["e_az"] = (np.max(dict_dl2[run]["az"]) - np.min(dict_dl2[run]["az"])) / 2
            dict_total["run"][run]["effective_time"] = dict_dl2[run]["effective_time"]

            counts, Iedges = np.histogram(np.log10(dict_dl2[run]["intensity"]), binsI)
            counts   = counts / dict_dl2[run]["effective_time"]

            dict_total["run"][run]["intensity_rates"] = counts

            dict_total["run"][run]["n_events"] = len(dict_dl2[run]["zd"])

        Icenters = (Iedges[1:] + Iedges[:-1]) / 2

        dict_total["general"]["intensity_bins"]         = Iedges
        dict_total["general"]["intensity_bins_centers"] = Icenters

        # Saving the object
        with open(fname_dict, 'wb') as f:
            pickle.dump(dict_total, f, pickle.HIGHEST_PROTOCOL)

Creating total dict --> dict_total_1.890_50.pkl
Creating total dict --> dict_total_1.956_50.pkl
Creating total dict --> dict_total_10.000_50.pkl
Creating total dict --> dict_total_2.026_50.pkl
Creating total dict --> dict_total_2.097_50.pkl
Creating total dict --> dict_total_2.171_50.pkl
Creating total dict --> dict_total_2.248_50.pkl
Creating total dict --> dict_total_2.327_50.pkl
Creating total dict --> dict_total_2.409_50.pkl
Creating total dict --> dict_total_2.495_50.pkl
Creating total dict --> dict_total_2.583_50.pkl
Creating total dict --> dict_total_2.674_50.pkl
Creating total dict --> dict_total_2.768_50.pkl
Creating total dict --> dict_total_2.866_50.pkl
Creating total dict --> dict_total_2.967_50.pkl
Creating total dict --> dict_total_3.072_50.pkl
Creating total dict --> dict_total_3.181_50.pkl
Creating total dict --> dict_total_3.293_50.pkl
Creating total dict --> dict_total_3.409_50.pkl
Creating total dict --> dict_total_3.530_50.pkl
Creating total dict --> dict_total_3.65

In [5]:
for double in a_lcs:

    # Energy LC limits
    e_lc_min, e_lc_max = double
    
    fname_dict_sed_lc = os.path.join(folder_dirs_sed_lc, f"dict_{e_lc_min:.3f}_{int(e_lc_max)}.pkl")
    
    if not os.path.exists(fname_dict_sed_lc):
        print(f"For integration from {e_lc_min:.4f}TeV to {int(e_lc_max)}TeV, job failed to launch")

For integration from 0.0100TeV to 50TeV, job failed to launch
For integration from 0.0104TeV to 50TeV, job failed to launch
For integration from 0.0107TeV to 50TeV, job failed to launch
For integration from 0.0111TeV to 50TeV, job failed to launch
For integration from 0.0115TeV to 50TeV, job failed to launch
For integration from 0.0119TeV to 50TeV, job failed to launch
For integration from 0.0123TeV to 50TeV, job failed to launch
For integration from 0.0128TeV to 50TeV, job failed to launch
For integration from 0.0132TeV to 50TeV, job failed to launch
For integration from 0.0137TeV to 50TeV, job failed to launch
For integration from 0.0141TeV to 50TeV, job failed to launch
For integration from 0.0146TeV to 50TeV, job failed to launch
For integration from 0.0152TeV to 50TeV, job failed to launch
For integration from 0.0157TeV to 50TeV, job failed to launch
For integration from 0.0163TeV to 50TeV, job failed to launch
For integration from 0.0168TeV to 50TeV, job failed to launch
For inte